In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca

In [53]:
def h1(x):
    return ca.vertcat(*[ca.norm_2(x[1]**2+x[2]**2-1),
                        ca.norm_2(x[0]-0)])
    

#def h2(x):
#    return ca.norm_2(x-np.array([0,1,0]))



In [67]:
#h1(np.array([1.1, 3.1, 1.1]))
x_sym = ca.SX.sym('x_sym',3)
h1_sym = h1(x_sym)
h2_sym = h2(x_sym)
h1_jac = ca.jacobian(h1_sym, x_sym)
h2_jac = ca.jacobian(h2_sym, x_sym)
h1_jac_fn = ca.Function('h1_jac', [x_sym],[h1_jac])
h2_jac_fn = ca.Function('h2_jac', [x_sym],[h2_jac])
print(h1_sym)
print(h2_sym)
print(h1_jac_fn(np.array([1, 1, 0.03])))

h1_jac_pinv = ca.pinv(h1_jac)
h1_jac_pinv_fn = ca.Function('h1_jac_pinv_fn', [x_sym], [h1_jac_pinv])

h2_jac_pinv = ca.pinv(h2_jac)
h2_jac_pinv_fn = ca.Function('h2_jac_pinv_fn', [x_sym], [h2_jac_pinv])


[fabs(((sq(x_sym_1)+sq(x_sym_2))-1)), fabs(x_sym_0)]
sqrt(((sq(x_sym_0)+sq((x_sym_1-1)))+sq(x_sym_2)))

[[00, 2, 0.06], 
 [1, 00, 00]]


In [85]:
#A*x=b
#J*lam=F

F = np.array([2, 1, 1])
x = np.array([2,0,1.3])
pi1_val = h1_jac_pinv_fn(x).full()
pi2_val = h2_jac_pinv_fn(x).full()

#print(h1_jac_fn(x))
#print(pi_val.size)
#print(F.size)
#print(h1_jac_fn(x).full().shape)

print(ca.norm_2(F-h1_jac_fn(x).T@(F@pi1_val)).full())
#print(ca.norm_2(F-h2_jac_fn(x)@(F@pi2_val)).full())


[[1.]]


In [18]:
### Rotation test
from rotation_helpers import *

#r = np.array([0, np.pi, 0])
r = np.array([-0.1, 0, 1.3])
r = ca.SX.sym('r', 3)


R = rotvec_to_rotation(r)
print(R)
r2 = rotation_to_rotvec(R)
print(r2)
R2 = rotvec_to_rotation(r2)
print(R-R2)

@1=sqrt(((sq(r_0)+sq(r_1))+sq(r_2))), @2=1e-09, @3=(@1<@2), @4=((@3?@2:0)+((!@3)?@1:0)), @5=(r_0/@4), @6=cos(@4), @7=(1-@6), @8=(r_1/@4), @9=(r_2/@4), @10=sin(@4), 
[[((sq(@5)*@7)+@6), (((@5*@8)*@7)-(@9*@10)), (((@5*@9)*@7)+(@8*@10))], 
 [(((@5*@8)*@7)+(@9*@10)), ((sq(@8)*@7)+@6), (((@8*@9)*@7)-(@5*@10))], 
 [(((@5*@9)*@7)-(@8*@10)), (((@8*@9)*@7)+(@5*@10)), ((sq(@9)*@7)+@6)]]
@1=sqrt(((sq(r_0)+sq(r_1))+sq(r_2))), @2=1e-09, @3=(@1<@2), @4=((@3?@2:0)+((!@3)?@1:0)), @5=(r_0/@4), @6=1, @7=cos(@4), @8=(@6-@7), @9=((sq(@5)*@8)+@7), @10=(r_1/@4), @11=((sq(@10)*@8)+@7), @12=(r_2/@4), @13=((sq(@12)*@8)+@7), @14=((@9+@11)+@13), @15=(fabs((@14+@6))<@2), @16=0.5, @17=(!@15), @18=3, @19=(!(fabs((@14-@18))<@2)), @20=sin(@4), @21=sqrt(((@18-@14)*(@6+@14))), @22=acos((@16*(@14-@6))), [(((@15?sqrt((@16*(@6+@9))):0)+(@17?(@19?(((((@10*@12)*@8)+(@5*@20))-(((@10*@12)*@8)-(@5*@20)))/@21):0):0))*@22), (((@15?sqrt((@16*(@6+@11))):0)+(@17?(@19?(((((@5*@12)*@8)+(@10*@20))-(((@5*@12)*@8)-(@10*@20)))/@21):0):0)